### 피처 
IRS 1년물, 10년물/ CRS 1년물, NDF 1월물, 국고채 1년물, 통안채 1년물, 스왑포인트

- IRS 1년물, 10년물 선정이유: 히트맵에서 IRS 1,2,3,5,10년물간의 상관관계를 봤을때  1,2년물/ 3,5,10년물끼리 다중공선성이 나와서 피처로 2개를 사용해도 상관없다고 판다  

### 환율예측 회귀식 
Yt = Yt-1 + MDF 변화량 + 스왑포인트 변화량 + IRS 1년물 변화율 + IRS 10년물 변화율 + CRS 1년물 변화율 + 국고채 1년물 변화율 + 통안채 1년물 변화율

In [19]:
import pandas as pd
import numpy as np

In [20]:
df = pd.read_excel("./xlsx/Join_data.xlsx")

In [21]:
df.columns

Index(['Unnamed: 0', 'DateTime', '종가_ex', '대비_ex', '증감률_ex', '1Y_Mid_irs',
       '1Y_전일비_irs', '2Y_Mid_irs', '2Y_전일비_irs', '3Y_Mid_irs', '3Y_전일비_irs',
       '5Y_Mid_irs', '5Y_전일비_irs', '10Y_Mid_irs', '10Y_전일비_irs', '1Y_Mid_crs',
       '1Y_전일비_crs', '2Y_Mid_crs', '2Y_전일비_crs', '3Y_Mid_crs', '3Y_전일비_crs',
       '5Y_Mid_crs', '5Y_전일비_crs', '10Y_Mid_crs', '10Y_전일비_crs', '국고1년',
       '국고3년', '국고5년', '국고10년', '통안364일', '통안2년', 'Bid_ndf', 'Ask_ndf',
       'Mid_ndf', '전일비_ndf', '1Y_베이시스', '2Y_베이시스', '3Y_베이시스', '5Y_베이시스',
       '10Y_베이시스', 'M1_스왑포인트'],
      dtype='object')

In [60]:
df

,Unnamed: 0,DateTime,종가_ex,대비_ex,증감률_ex,1Y_Mid_irs,1Y_전일비_irs,2Y_Mid_irs,2Y_전일비_irs,3Y_Mid_irs,...,Bid_ndf,Ask_ndf,Mid_ndf,전일비_ndf,1Y_베이시스,2Y_베이시스,3Y_베이시스,5Y_베이시스,10Y_베이시스,M1_스왑포인트
0,0,2012-08-01,1126.5,-4.1,-0.36,2.850,-0.03,2.740,-0.03,2.740,...,1132.0,1132.5,1132.25,NaN,-76.0,-92.5,-93.0,-105.0,-108.0,260
1,1,2012-08-02,1131.7,5.2,0.46,2.820,-0.03,2.690,-0.05,2.690,...,1133.5,1134.5,1134.00,1.75,-74.0,-84.5,-84.0,-96.0,-99.0,250
2,2,2012-08-03,1134.8,3.1,0.27,2.790,-0.03,2.660,-0.03,2.660,...,1137.5,1138.5,1138.00,4.00,-72.0,-83.0,-83.0,-101.0,-112.0,240
3,3,2012-08-06,1129.0,-5.8,-0.51,2.810,0.02,2.680,0.02,2.680,...,1128.0,1129.0,1128.50,-9.50,-74.0,-87.5,-88.0,-107.0,-117.0,240
4,4,2012-08-07,1128.8,-0.2,-0.02,2.820,0.01,2.680,0.00,2.680,...,1128.5,1129.5,1129.00,0.50,-73.0,-86.0,-88.0,-115.0,-127.0,250
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2455,2455,2022-07-25,1313.7,0.7,0.05,3.165,-0.04,3.235,-0.08,3.205,...,1309.7,1310.0,1309.85,3.35,-61.5,-50.5,-49.5,-61.5,-60.5,-60
2456,2456,2022-07-26,1307.6,-6.1,-0.47,3.155,-0.01,3.215,-0.02,3.175,...,1310.9,1311.1,1311.00,1.15,-59.5,-51.5,-49.5,-60.5,-59.5,-70
2457,2457,2022-07-27,1313.3,5.7,0.43,3.145,-0.01,3.165,-0.05,3.115,...,1310.4,1310.6,1310.50,-0.50,-57.5,-47.5,-44.5,-55.5,-54.5,-55
2458,2458,2022-07-28,1296.1,-17.2,-1.33,3.175,0.03,3.205,0.04,3.165,...,1305.8,1306.2,1306.00,-4.50,-56.5,-47.5,-45.5,-57.5,-59.5,-65


In [97]:
# 인덱스를 DateTime으로 변경

df_2= df.set_index('DateTime')
df_2

,Unnamed: 0,종가_ex,대비_ex,증감률_ex,1Y_Mid_irs,1Y_전일비_irs,2Y_Mid_irs,2Y_전일비_irs,3Y_Mid_irs,3Y_전일비_irs,...,Bid_ndf,Ask_ndf,Mid_ndf,전일비_ndf,1Y_베이시스,2Y_베이시스,3Y_베이시스,5Y_베이시스,10Y_베이시스,M1_스왑포인트
DateTime,,,,,,,,,,,,,,,,,,,,,
2012-08-01,0,1126.5,-4.1,-0.36,2.850,-0.03,2.740,-0.03,2.740,-0.03,...,1132.0,1132.5,1132.25,NaN,-76.0,-92.5,-93.0,-105.0,-108.0,260
2012-08-02,1,1131.7,5.2,0.46,2.820,-0.03,2.690,-0.05,2.690,-0.05,...,1133.5,1134.5,1134.00,1.75,-74.0,-84.5,-84.0,-96.0,-99.0,250
2012-08-03,2,1134.8,3.1,0.27,2.790,-0.03,2.660,-0.03,2.660,-0.03,...,1137.5,1138.5,1138.00,4.00,-72.0,-83.0,-83.0,-101.0,-112.0,240
2012-08-06,3,1129.0,-5.8,-0.51,2.810,0.02,2.680,0.02,2.680,0.02,...,1128.0,1129.0,1128.50,-9.50,-74.0,-87.5,-88.0,-107.0,-117.0,240
2012-08-07,4,1128.8,-0.2,-0.02,2.820,0.01,2.680,0.00,2.680,0.00,...,1128.5,1129.5,1129.00,0.50,-73.0,-86.0,-88.0,-115.0,-127.0,250
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-07-25,2455,1313.7,0.7,0.05,3.165,-0.04,3.235,-0.08,3.205,-0.07,...,1309.7,1310.0,1309.85,3.35,-61.5,-50.5,-49.5,-61.5,-60.5,-60
2022-07-26,2456,1307.6,-6.1,-0.47,3.155,-0.01,3.215,-0.02,3.175,-0.03,...,1310.9,1311.1,1311.00,1.15,-59.5,-51.5,-49.5,-60.5,-59.5,-70
2022-07-27,2457,1313.3,5.7,0.43,3.145,-0.01,3.165,-0.05,3.115,-0.06,...,1310.4,1310.6,1310.50,-0.50,-57.5,-47.5,-44.5,-55.5,-54.5,-55


In [104]:
# ':' -> 모든 행 가져오는것
# loc 컬럼 추출

df_3 = df.loc[:, ['DateTime', '종가_ex']]
df_3

,DateTime,종가_ex
0,2012-08-01,1126.5
1,2012-08-02,1131.7
2,2012-08-03,1134.8
3,2012-08-06,1129.0
4,2012-08-07,1128.8
...,...,...
2455,2022-07-25,1313.7
2456,2022-07-26,1307.6
2457,2022-07-27,1313.3
2458,2022-07-28,1296.1


In [134]:
df_5 = df_3.append({'DateTime': '2012-07-31','종가_ex': '1130.6'}, ignore_index=True)
df_5


C:\Users\hyeok\AppData\Local\Temp\ipykernel_20852\2046892599.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_5 = df_3.append({'DateTime': '2012-07-31','종가_ex': '1130.6'}, ignore_index=True)


,DateTime,종가_ex
0,2012-08-01,1126.5
1,2012-08-02,1131.7
2,2012-08-03,1134.8
3,2012-08-06,1129.0
4,2012-08-07,1128.8
...,...,...
2456,2022-07-26,1307.6
2457,2022-07-27,1313.3
2458,2022-07-28,1296.1
2459,2022-07-29,1299.1


In [135]:
df_6 = df_5.shift(1)
df_6

,DateTime,종가_ex
0,NaN,NaN
1,2012-08-01,1126.5
2,2012-08-02,1131.7
3,2012-08-03,1134.8
4,2012-08-06,1129.0
...,...,...
2456,2022-07-25,1313.7
2457,2022-07-26,1307.6
2458,2022-07-27,1313.3
2459,2022-07-28,1296.1


In [137]:
df_6.loc[0,'DateTime'] = '2012-07-31'
df_6.loc[0,'종가_ex'] = '1130.6'
df_6

,DateTime,종가_ex
0,2012-07-31,1130.6
1,2012-08-01,1126.5
2,2012-08-02,1131.7
3,2012-08-03,1134.8
4,2012-08-06,1129.0
...,...,...
2456,2022-07-25,1313.7
2457,2022-07-26,1307.6
2458,2022-07-27,1313.3
2459,2022-07-28,1296.1


In [140]:
df_7 = df_6.drop(df_6.index[2460])
df_7

,DateTime,종가_ex
0,2012-07-31,1130.6
1,2012-08-01,1126.5
2,2012-08-02,1131.7
3,2012-08-03,1134.8
4,2012-08-06,1129.0
...,...,...
2455,2022-07-22,1313.0
2456,2022-07-25,1313.7
2457,2022-07-26,1307.6
2458,2022-07-27,1313.3


In [141]:
# 환율전날종가
df_tt = df_7